In [ ]:
import argparse, datetime, json
import data_cleaning as clean
import fitting as fit
import numpy as np
import os
import pandas as pd
import stat_metrics as stat
import utils

DATA_FILENAME = 'FLE7_2020.csv'
N_ITERS       = 1500

# Load behavioral data from CSV
data = pd.read_csv(DATA_FILENAME, delimiter=';', header=None).values
num_subjects = data.shape[0]

# Define the x-axis (0:pi/4:2*pi)
xax = utils.X_RANGE

# Sort properly the condition, according to the X_RANGE
data = clean.sort_condition(data)

# Flatten the data to 9x32 for fitting
data = data.reshape(num_subjects, -1)  # 9x32 matrix
data_tmp = np.zeros((data.shape[0]+1, data.shape[1]))
data_tmp[:num_subjects, :] = data   
# At the bottom of the original data matrix, added the mean across the subjects
data_tmp[-1, :] = np.nanmean(data, axis = 0) 
data = data_tmp 

# Prepare X_data (same values for all subjects, repeated)
xax = np.tile(xax, 4)
x_data = np.ones((data.shape[0], len(xax)))*xax

# Fit the data, either sub by sub and all together
betas, fit_quality, prediction_subs              = fit.fit_data_sub_B_sub(data, x_data)
beta_global, fit_quality_global, prediction_glob = fit.fit_all_together(data[:-1, :], x_data[:-1, :])  
 
utils.fit_result_outcome(betas, fit_quality, beta_global, fit_quality_global, filename = 'model_fitting_results.csv')

# Break fitting sub by sub
mses_err_sub, rsqr_err_sub = stat.break_fit(data, prediction_subs, N_ITERS)
# Break fitting global
mses_err_glob, rsqr_err_glob = stat.break_fit(data[:-1, :].ravel(), prediction_glob.ravel(), N_ITERS*10)


### Obtain CI half-width 95% and store it

In [ ]:
x_preds_, y_preds_, deltas_ = fit.get_confidence_interval(data, x_data, 
                                                          betas, beta_global, 
                                                          prediction_subs, 
                                                          alpha = .05, eps = 1e-5)

sub_names = utils.SUB_NAMES + ['AllTogether']
utils.prediction_outcome(x_preds_, y_preds_, deltas_, sub_names)

### Visualize fittings functions and parameters

In [ ]:
import visualization_fit as visualize

visualize.visualize_fit_n_raw_sub(data, x_data, betas, utils.SUB_NAMES, 'fit_allSubs')
visualize.visualize_fit_n_raw_allTogether(data[:-1, :], x_data[:-1, :], beta_global, filename = 'fit_allTogether')

print(f'Betas for average across subjects {betas[-1]}')
print(f'Betas for all together fit {beta_global}')

visualize.visualize_betas(betas, filename = 'betas_subs', betas_to_plot = ['Fovea', 'HM', 'VM', 'Eccentricity'])


visualize.visualize_hists(mses_err_sub, 
                          rsqr_err_sub, 
                          fit_quality, 
                          filename = 'random_permutation_distributions_subbysub')

visualize.visualize_hists(np.array([mses_err_glob]), 
                          np.array([rsqr_err_glob]), 
                          fit_quality_global, 
                          sub_names = ['All Together'],                          
                          filename = 'random_permutation_distributions_allTogether')


In [ ]:
p_allTog, p_ttest_allTog, ps_single_sub, ps_single_sub_t = stat.get_pvalues([mses_err_glob, rsqr_err_glob], 
                                                                            [mses_err_sub, rsqr_err_sub],
                                                                            fit_quality_global, fit_quality)

In [ ]:
utils.stat_outcome(ps_single_sub, ps_single_sub_t, 
                   p_allTog, 
                   p_ttest_allTog, 
                   fit_quality_global, fit_quality, filename = 'breakfit_random_perm_quality_results')

## Fit of randomly permuted data 

In [ ]:
import datetime 

betas_sub_shuffled       = np.zeros((utils.N_ITERS, data.shape[0], len(utils.BETA_INIT)))
fit_quality_sub_shuffled = list()
start_time   = datetime.datetime.now().replace(microsecond=0)

betas_allTogether_shuffled       = np.zeros((utils.N_ITERS, len(utils.BETA_INIT)))
fit_quality_allTogether_shuffled = list()
start_time   = datetime.datetime.now().replace(microsecond=0)

for i in range(utils.N_ITERS):
    shuffled_data = np.array([np.random.permutation(row) for row in data])    
    betas_sub_shuffled[i, :, :], a, _ = fit.fit_data_sub_B_sub(shuffled_data, x_data, flag_print = False)
    fit_quality_sub_shuffled.append(a)
    
    shuffled_data = np.array([np.random.permutation(row) for row in data])    
    betas_allTogether_shuffled[i, :], a, _ = fit.fit_all_together(shuffled_data[:-1, :], x_data[:-1, :], flag_print = False)
    fit_quality_allTogether_shuffled.append(a)
    
    if i % 50 == 0:
        tmp_rmse = fit_quality_allTogether_shuffled[-1][0][0]
        tmp_rsqr = fit_quality_allTogether_shuffled[-1][0][1]
        print(f"--------------------------------------------------------------------------------")
        print(f"Iteration {i + 1}")
        print(f"RMSE: {tmp_rmse:.4f}, \
              R²: {tmp_rsqr:.4f}")        
            
    
        tmp_rmse = fit_quality_sub_shuffled[-1][0][0]
        tmp_rsqr = fit_quality_sub_shuffled[-1][0][1]

        print(f"RMSE sub: {tmp_rmse:.4f}, \
              R² sub: {tmp_rsqr:.4f}")        
        
fit_quality_sub_shuffled = np.array([[j[0], j[1]] for i in fit_quality_sub_shuffled for j in i])        
fit_quality_sub_shuffled = fit_quality_sub_shuffled.reshape(1500, 10, 2).transpose(1, 0, 2)

fit_quality_allTogether_shuffled = np.array([[i[0][0], i[0][1]]for i in fit_quality_allTogether_shuffled])

print(f'Fitting on randomly permuted data: {str(datetime.datetime.now().replace(microsecond=0)-start_time)}')


In [ ]:
visualize.visualize_hists(np.squeeze(fit_quality_sub_shuffled[:, :, 0]), 
                          np.squeeze(fit_quality_sub_shuffled[:, :, 1]), 
                          fit_quality, 
                          filename = 'random_permutation_fits_subbysub')

In [ ]:
visualize.visualize_hists(fit_quality_allTogether_shuffled[:, 0].reshape(1, -1), 
                          fit_quality_allTogether_shuffled[:, 1].reshape(1, -1), 
                          fit_quality_global, 
                          sub_names = ['All Together'],                          
                          filename = 'random_permutation_fits_allTogether')


In [ ]:
p_allTog, p_ttest_allTog, ps_single_sub, ps_single_sub_t = stat.get_pvalues([np.squeeze(fit_quality_allTogether_shuffled[:, 0]), np.squeeze(fit_quality_allTogether_shuffled[:, 1])], 
                                                                            [np.squeeze(fit_quality_sub_shuffled[:, :, 0]), np.squeeze(fit_quality_sub_shuffled[:, :, 1])],
                                                                            fit_quality_global, fit_quality)

In [ ]:
utils.stat_outcome(ps_single_sub, ps_single_sub_t, 
                   p_allTog, p_ttest_allTog, 
                   fit_quality_global, fit_quality, filename = 'fit_random_perm_quality_results')

In [ ]:
not_signif_subs = {n: utils.SUB_NAMES[n] for n, (i, j) in enumerate(ps_single_sub) if ((i>0.05) and (j>0.05)) }
not_signif_subs

In [ ]:
data_signif = np.array([i for n, i in enumerate(data) if n not in list(not_signif_subs.keys())])

In [ ]:
# Fit the data, either sub by sub and all together
betas_signif, fit_quality_signif, prediction_subs_signif              = fit.fit_data_sub_B_sub(data_signif, x_data[:len(data_signif), :])
beta_global_signif, fit_quality_global_signif, prediction_glob_signif = fit.fit_all_together(data_signif[:-1, :], x_data[:(len(data_signif)-1), :])  

visualize.visualize_fit_n_raw_allTogether(data_signif[:-1, :], 
                                          x_data[:(len(data_signif)-1), :], 
                                          beta_global_signif, 
                                          filename = 'fit_allTogether_only_Signif')

print(f'Betas for average across subjects {betas_signif[-1]}')
print(f'Betas for all together fit {beta_global_signif}')

visualize.visualize_betas(betas, filename = 'betas_subs_only_Signif', 
                          betas_to_plot = ['Fovea', 'HM', 'VM', 'Eccentricity'], 
                          not_significant_subs = [utils.SUB_ENUMERATION[i] for i in list(not_signif_subs.keys())])


In [ ]:
beta_global - beta_global_signif

In [ ]:
fit_quality_global_signif

In [ ]:
tmp_row = [fit_quality_global_signif[0][0], 
           fit_quality_global_signif[0][1], 
           beta_global_signif[0], 
           beta_global_signif[1], 
           beta_global_signif[2],
           beta_global_signif[3], 
           beta_global_signif[4]]
utils.append_row_to_csv(tmp_row, new_index='Only Significant', filename='model_fitting_results.csv')